In [26]:
"""
Usage: python prestart_analysis.py rr_number race_number start end interval_len
"""


import pypyodbc
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import matplotlib.patches as mpatches
import time
import sys

# connect to the database using pypyodbc
db_host = "192.168.100.35"
db_name = "AC35"
db_user = "dbreader"
db_password = "dbr3ader"

connection_string='DSN=MYMSSQL;DATABASE=AC35;UID=dbreader;PWD=dbr3ader'

db = pypyodbc.connect(connection_string)
crsr = db.cursor()

"""
this data needs to be hardcoded in everyday in order to ensure that the program
can access the database properly

rr_i = round robin i
"""
rr1 = {1: ["USA", "FRA"], 2: ["SWE", "JPN"], 3: ["NZL", "FRA"],
       4: ["SWE", "GBR"], 5: ["USA", "NZL"], 6: ["JPN", "GBR"],
       7: ["SWE", "FRA"], 8: ["USA", "GBR"], 9: ["NZL", "JPN"],
       10: ["USA", "SWE"], 11: ["NZL", "GBR"], 12: ["USA", "JPN"],
       13: ["FRA", "GBR"], 14: ["SWE", "NZL"], 15: ["JPN", "FRA"]}

rr2 = {1: ["SWE", "NZL"], 2: ["USA", "FRA"], 3: ["SWE", "GBR"]}

round_robins = [rr1, rr2]

def get_competitor_info(rr_num, race_number):
    """
    function to return the relevant competitors in each round robin

    rr_num = round robin number (int)
    race_number = (int)
    """
    return(round_robins[rr_num - 1][race_number])

# map ACEA names to database names
db_names = {"NZL": "ETNZ", "USA": "ORACLE", "SWE": "ARTEMIS", "JPN": "JAPAN",
            "GBR": "GBR", "FRA": "FRANCE"}

def execute_query(q_string):
    """
    executes query on the database

    q_string = query (string)
    """
    return crsr.execute(q_string).fetchall()

def get_race_start(rr_number, race_number):
    """
    returns TS corresponding to the warning signal (3 minutes before the gun)
    of the relevant race

    return type (long)
    """
    competitors = ",".join(get_competitor_info(rr_number, race_number))
    q = ("select startTS from [AC35].a.ranges where rangenotes "
         "like '%RACE {0} %({1})% Warning,%'").format(race_number, competitors)
    return execute_query(q)[0][0]

def get_speeds(rr_number, race_number, start, end):
    """
    rr_number: int corresponding to the round robin number
    race_number: int corresponding to the relevant race number
    start: seconds before the start, should be a negative number,
           must be in (-180, 0)
    end: seconds after the start, should be a positve number, must be in
         (0, End of Race)

    returns: Pandas DataFrame with timestep and sogs corresponding to both
             teams
    """
    race_start = get_race_start(rr_number, race_number)
    lb = race_start + (180 + start) * 1000
    ub = race_start + (180 + end) * 1000

    boat1, boat2 = get_competitor_info(rr_number, race_number)
    db1 = db_names[boat1]
    db2 = db_names[boat2]

    query = ("select A.TS, A.V '{0} SOG', B.V '{1} SOG' from ("
             "(select SID, TS, V from [AC35].[d].[ACEA.{0}.SOG] where ts between {2} and {3}) A "
             "inner join "
             "(select SID, TS, V from [AC35].[d].[ACEA.{1}.SOG] where ts between {2} and {3}) B "
             "on A.TS = B.TS) "
             "order by TS asc"
            ).format(db1, db2, lb, ub)

    columns = ["Timestep", db1 + " SOG", db2 + " SOG"]
    df = pd.DataFrame(execute_query(query), columns=columns)
    df['Timestep'] = (df['Timestep'] - (race_start + 180 *1000)) / 1000
    return df


# In[29]:

def get_positions(rr_number, race_number, start, end):
    """
    rr_number: int corresponding to the round robin number
    race_number: int corresponding to the relevant race number
    start: seconds before the start, should be a negative number,
           must be in (-180, 0)
    end: seconds after the start, should be a positve number, must be in (0, )

    returns: Pandas DataFrame with TimeStep, x,y, and COG for each boat
    """
    race_start = get_race_start(rr_number, race_number)
    lb = race_start + (180 + start) * 1000
    ub = race_start + (180 + end) * 1000

    boat1, boat2 = get_competitor_info(rr_number, race_number)
    db1 = db_names[boat1]
    db2 = db_names[boat2]

    q = ("select A.TS, A.V '{0} X', B.V '{0} Y', C.V '{0} COG', D.V '{1} X', E.V '{1} Y', F.V '{1} COG'from ("
         "(select TS, V from [AC35].[d].[ACEA.{0}.calculated.x] where ts between {2} and {3}) A "
         "inner join "
         "(select TS, V from [AC35].[d].[ACEA.{0}.calculated.y] where ts between {2} and {3}) B "
         "on A.TS = B.TS "
         "inner join "
         "(select TS, V from [AC35].[d].[ACEA.{0}.COG] where ts between {2} and {3}) C "
         "on C.TS = B.TS "
         "inner join "
         "(select TS, V from [AC35].[d].[ACEA.{1}.calculated.x] where ts between {2} and {3}) D "
         "on D.TS = C.TS "
         "inner join "
         "(select TS, V from [AC35].[d].[ACEA.{1}.calculated.y] where ts between {2} and {3}) E "
         "on E.TS = D.TS "
         "inner join "
         "(select TS, V from [AC35].[d].[ACEA.{1}.COG] where ts between {2} and {3}) F "
         "on F.TS = E.TS) "
         "order by TS asc").format(db1, db2, lb, ub)

    columns = ["Timestep", db1 + " X", db1 + " Y", db1 + " COG", db2 + " X",
               db2 + " Y", db2 + " COG"]
    df = pd.DataFrame(execute_query(q), columns=columns)
    df['Timestep'] = (df['Timestep'] - (race_start + 180 *1000)) / 1000
    return df

def plot_speeds(rr_number, race_number, start, end):
    """
    rr_number: int corresponding to the round robin number
    race_number: int corresponding to the relevant race number
    start: seconds before the start, should be a negative number,
           must be in (-180, 0)
    end: seconds after the start, should be a positve number, must be in (0, )

    returns: None, plots graph of SOG's for each boat over the relevant interval
    """
    dat = get_speeds(rr_number, race_number, start, end)
    b1, b2 = [db_names[i] for i in get_competitor_info(rr_number, race_number)]
    plt.figure(figsize=(8,4))
    plt.title("Round Robin {} Race {} Prestart SOG".format(rr_number,
                                                           race_number))
    plt.plot(dat["Timestep"], dat[b1 + " SOG"], label=b1)
    plt.plot(dat["Timestep"], dat[b2 + " SOG"],label=b2)
    plt.ylabel("Speed in Knots")
    plt.xlabel("Time")
    plt.legend(bbox_to_anchor=(1.35, .55))
    plt.savefig('Speeds Round Robin {} Race Number {} {} to {}.png'.format(rr_number, race_number,
                start, end), bbox_inches='tight')
    # plt.show()
    # plt.close()

def get_marks(rr_number, race_number):
    """
    returns: np array where of rows corresponding to start marks and
             columns to x, y coordinates for each
    """
    time = get_race_start(rr_number, race_number)
    marks = ["SL1", "SL2"]
    vals = ['x', 'y']
    res = [range(2) for x in range(2)]
    for i,mark in enumerate(marks):
        for j,val in enumerate(vals):
            q = "select TOP 1 V from [AC35].[d].[ACEA.{0}.calculated.{1}] where TS between {2} and {2} + 10000".format(mark, val, time)
            res[i][j] = execute_query(q)[0][0]
    q = "select V from [AC35].[d].[ACEA.SL1.TrueWindSpeed] where TS between {0} + 90 * 1000 and {0} + 3 * 60 * 1000".format(time)
    wind_speed = np.mean(np.array(execute_query(q))[:,0])
    return (np.array(res), wind_speed)


In [27]:
get_marks(1, 1)

(array([[ 1720.74564315,  4479.17556276],
        [ 1903.30009896,  4556.04369891]]), 11.908534086503849)

In [11]:
get_race_start(1,1)

1495904699277L